## Geolocated clustering and sequence mining with Spark, Cassandra, and Scala

Today, geolocated data is available in a number of domains, ranging from healthcare to financial markets to social services. In all these domains, extracting patterns and detecting clusters and sequences of events from data has very concrete business outcomes. 

Cluster analysis on geolocated events is a key step for understanding users' behavior and it is essential for a more personalized customer experience, but also to prevent fraud and cyber attacks. By extracting sequences of events, we can determine how various venues, products, and services are connected; and provide a very detailed analysis about the popularity of certain items which conversely yields to better user recommendations and richer and more meaningful UI interactions.

As more data gets ingested or produced via digital services, it’s key to perform this sort of analytics at scale. In the open source space, technologies such as Spark and Cassandra are definitely instrumental to implement and execute modern data pipelines at scale.


#### Synopsis

In this Oriole, we will explore a number of techniques for understanding users' behavior: We will look at process mining to understand the _sequence_ of events registered by users, and apply graph analytics to detect the most popular venues in town. Also, we will look at how to sketch a geofencing, location-based alert service by applying the DBSCAN clustering algorithm to the geolocated events. 

We will follow these steps:

  - Data extraction from Cassandra
  - Process mining using Spark RDDs
  - Graph analytics with Spark GraphFrames
  - Clustering for geolocated data with DBSCAN
  
If you are interested in a general overview of descriptive analytics, histograms, and pattern detection, please have a look at the companion Oriole notebook "Anomaly detection and pattern extraction with Spark, Cassandra and Scala", also available on Safari.


#### References and datasets

For this analysis we are going to use the Gowalla dataset [1]. The Gowalla dataset consists of a table of events registered by anonymized users. Each event registers a user checking into a geolocated venue at a specific timestamp. The dataset is available at <a href="https://snap.stanford.edu/data/loc-gowalla.html" target="_blank">https://snap.stanford.edu/data/loc-gowalla.html</a>

A number of venues in this demo have been tagged with an actual name, thanks to the <a href="https://code.google.com/archive/p/locrec/" target="_blank">https://code.google.com/archive/p/locrec/</a> project (now archived). The project is being developed in the context of the SInteliGIS project financed by the Portuguese Foundation for Science and Technology (FCT) through project grant PTDC/EIA-EIA/109840/2009.

[1] E. Cho, S. A. Myers, J. Leskovec. Friendship and Mobility: Friendship and Mobility: User Movement in Location-Based Social Networks ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2011.



#### Setup

This notebook is running Scala code and interfaces to a Spark cluster using the <a href="https://toree.incubator.apache.org/" target="_blank">Apache Toree</a> project; furthermore, Spark reads the data from Cassandra tables. Spark interfaces to Cassandra via the <a href="https://github.com/datastax/spark-cassandra-connector/" target="_blank">Cassandra-Spark connector</a>.

At the time of compiling this notebook, Spark 1.6.1 and Cassandra 3.5 were used. Below, you will find the command to install the Spark - Scala kernel on Jupyter. More instructions on this topic are available on the Apache Toree <a href="https://toree.incubator.apache.org/" target="_blank">website</a> and <a href="https://github.com/apache/incubator-toree" target="_blank">github pages</a>.

This particular Oriole is special in many ways. For instance, as you can see above from the Toree kernel configuration, next to the ability to connect to Cassandra, it adds <a href="https://graphframes.github.io/" target="_blank">GraphFrames</a> as an extra custom Spark package to the mix. 

```
sudo jupyter-toree install --spark_home=${SPARK_HOME} 
--spark_opts='--packages com.datastax.spark:spark-cassandra
-connector_2.10:1.6.0,graphframes:graphframes:0.1.0-spark1.6 
--conf spark.cassandra.connection.host=localhost '
```

_Please allow a few seconds for the next cell to complete.  
It will start a Single-Node Spark context on the Oriole container, and connect it to the Oriole Scala code cells._

In [ ]:
// Scala version
sc.version

### Connecting to Cassandra

In [ ]:
//sql context
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.functions._

val sqlContext  = new SQLContext(sc)
import sqlContext.implicits._

In [ ]:
// spark-cassandra connector
import com.datastax.spark.connector._
import com.datastax.spark.connector.cql._

import org.apache.spark.sql.cassandra.CassandraSQLContext
val cc = new CassandraSQLContext(sc)

#### SQL queries with Spark and Cassandra

Cassandra is exposed via a SQL context, so there is no need to learn a separate syntax as Spark will map the query to the available features of the underlying storage system. Below, you will see a simple query accessing the name and the id of venues from a Cassandra table. Also remember that SQL statements are _staged_ but not _executed_ until some actual <a href="http://spark.apache.org/docs/latest/programming-guide.html#actions" target="_blank">actions</a> needs to be computed. Some examples of actions are: **count**(), **first**(), **collect**().

#### Dataset: Venues

First, let's have a quick look at the venues. They are stored in the Cassandra table `lbsn.venues`. 

In [ ]:
val venues   = cc.sql("select vid, name from lbsn.venues").as("venues")

Let's start by counting the number of venues, and selecting vid `12525`, to get a feeling about how Spark works and getting some facts about the dataset.
Feel free to modify the cells below to gain insight on the venue data. For instance, try the `take()` and the `show()` Spark DataFrame methods.

In [ ]:
venues.count()

In [ ]:
venues.where("vid = 12525").first()

#### Executing queries

Spark and Cassandra work together when it comes to executing queries. If the query can be executed directly on the database, Spark will offload the query to Cassandra. However, not all queries can be fully performed on Cassandra; that's where the combination Spark-Cassandra gets really handy. For instance, when executing joins, Spark will partition and plan the query _pushing down_ what can be done in Cassandra and then perform the rest of the query in Spark. 

More information can be found on Cassandra documentation about <a href="http://docs.datastax.com/en/datastax_enterprise/5.0/datastax_enterprise/spark/sparkSqlOverview.html" target="_blank">using Spark SQL to query data</a> or on the <a href="https://github.com/datastax/spark-cassandra-connector" target="_blank">Cassandra Spark Connector</a> pages.

#### Joining Cassandra tables with Spark

The query shown below filters out those events which were registered in the New York City area. Because filtering in Cassandra cannot by done directly (lat/lon columns which are not indexed in this example), this specific query will first move the data from Cassandra to Spark, and then will perform the filtering in Spark. 

In general, it's good practice to push down and filter as much data as early as possible. This practice keeps the throughput low and minimizes the data transferred from one system to the other.

#### Dataset: Events

Let's have a quick look at the events. They are stored in the Cassandra table `lbsn.events`. 

In [ ]:
val events   = cc.sql("""select ts, uid, lat, lon, vid from lbsn.events where
                            lon>-74.2589 and lon<-73.7004 and 
                            lat> 40.4774 and lat< 40.9176
                      """).as("events").orderBy("uid", "ts").repartition(8, $"uid").cache()

In [ ]:
events.printSchema()

As you can see above, Spark DataFrame can extract the schema from the Cassandra-Spark SQL query. In this particular case, the mapping of Cassandra types to Spark DataFrame types is performed by the <a href="https://github.com/datastax/spark-cassandra-connector" target="_blank">Cassandra-Spark connector</a>. Each event consists of a timestamp, the user and venue id, and the location of the event (latitude and longitude).


#### Explore the events data

Let's start by looking at some recorded events. A quick way to do so is with the `show()` DataFrame Spark method.
Feel free to modify the cells below to gain insight on the venue data. For instance, try the `take()`, `show()`, `count()` Spark DataFrame methods.

In [ ]:
events.show(5)

Before diving into anomaly detection of geolocated data, let's perform some more basic queries.  
Below, is shown how to count events registered by user `uid=0`.

In [ ]:
// User 0: how many check-ins?

events.where("uid=0").count()

### Process mining with Spark RDDs

The first step, in order to do process mining, is to collect sequences of events. In particular, the following code will take chronologically consecutive events and bundle them in pairs for a specific user. These pairs consist of two venue ids, namely source and destination, defining where each user is coming from and going to, respectively. 

The steps in the following code are:

  - Convert the DataFrame to an RDD
  - Select uid as the key for the PairRDD
  - Reshape the PairRDD from "tall" to "wide"
  - Sort chronologically all the checked-in venues for each user
  - Extract pairs from each sequence of checked-in venues per user
  - Reshape the PairRDD from "wide" to "tall" again
  - Convert the PairRDD back to a DataFrame

In [ ]:
// process mining
val g_df = events.
  select($"ts", $"uid", $"vid").
  rdd.
  map(row => (row.getLong(1), List( (row.getTimestamp(0), row.getLong(2)) ))).
  reduceByKey(_ ++ _).
  mapValues( x =>
    x.sortWith(_._1.getTime < _._1.getTime).
      map(_._2)
  ).
  mapValues(_.sliding(2).toList).
  flatMap(_._2).
  map(
    _ match {
      case List(a, b) => Some((a, b))
      case _ => None
  }).
  flatMap(x => x).
  toDF("src", "dst").
  repartition(8,$"src").
  cache()

This newly created DataFrame is used to create a graph, where the nodes are the venues and the edges are connections of users checking-in from one venue to the next. Let's do a bit of pruning on this graph by removing self-referencing nodes, and by filtering out nodes which are not connected at all; as shown below in the following two cells.

In [ ]:
val edges_df = g_df.
  groupBy($"src",$"dst").
  count().
  select($"src",$"dst").
  filter($"src" !== $"dst")

edges_df.show(5)

In [ ]:
val nodes_df = edges_df.
  select($"src").
  unionAll(edges_df.select($"dst")).
  distinct().
  toDF("id")

nodes_df.show(5)

### Venues graph

Congratulations! You have just constructed a graph, where venues are connected to other venues according to a sequence of events recorded by users. Let's have a quick look at the size of this graph. By the way, for those of you new to Scala, you can embed code in a Scala string using the `${ ... }` construct as shown below.

### ESZTI COMMENT - need to put in a highlight here saying "Run this cell" or something because he talks about the output without actually indicating to run the cell.

In [ ]:
println(s"# nodes (venues)      = ${nodes_df.count()}")
println(s"# edges (connections) = ${edges_df.count()}")

You can convert two Spark DataFrames, one for the nodes and the other for the edges as a GraphFrame g, as shown below.

In [ ]:
import org.graphframes.GraphFrame

val v = nodes_df.orderBy("id")
val e = edges_df.orderBy("src", "dst")

val g = GraphFrame(nodes_df, edges_df)

#### Graph: Node degree analysis

Let's start a basic graph analysis by looking first at in-degree. The in-degree is the property which counts the number of incoming edges for each node of the graph. We can use in-degree to sort which venues are receiving most incoming connections from other venues. This is one of the easiest ways to see which venues are popular. Let's have a look at which nodes/venues in the graph have at least 50 incoming connections, and display this list.

In [ ]:
val vertices_indeg = g.inDegrees.filter($"inDegree">50).sort(desc("inDegree"))

In [ ]:
val popular_venues = vertices_indeg.
    join(venues, vertices_indeg("id") === venues("vid"), "inner").
    sort($"inDegree".desc).
    select("vid", "inDegree", "name")
    
println(s"Top in-degree nodes: ${ popular_venues.count()}")
popular_venues.show(10, false)

Most transportation hubs would probably will be the most pospular according to this metric. But what about other ways of measuring the popularity of a venue?

#### Graph filtering

Before moving on to page rank, let's make the graph smaller. For the sake of performance and fast execution of the next cells, we are limiting the analysis to only those venues with at least 50 incoming connections (in-degree).

In [ ]:
val vertices  = vertices_indeg.select("id")

val edges_src = edges_df.join(vertices, vertices("id")  === edges_df("src"), "inner").select("src", "dst")
val edges     = edges_src.join(vertices, vertices("id") === edges_src("dst"), "inner").select("src", "dst")

println(s"graph: vertices=${vertices.count()} edges=${edges.count()}")

#### Graph and PageRank

The above node and edge DataFrames describes how users are moving from venue to venue. We can now calculate which venues attract more users. This can be done using the PageRank algorithm. The PageRank algorithm outputs a probability distribution used to represent the likelihood that a person randomly walking in the city will arrive at any particular venue. This analysis can be executed in Spark using <a href="http://graphframes.github.io/" targe="_blank">GraphFrames</a>. GraphFrames is a package for Apache Spark which provides DataFrame-based graph analytics, including the PageRank algorithm.

In [ ]:
val v = vertices.orderBy("id").repartition(8, $"id").cache()
val e = edges.orderBy("src", "dst").repartition(8, $"src", $"dst").cache()

In [ ]:
val g = GraphFrame(v, e)
val results = g.pageRank.resetProbability(0.05).maxIter(10).run()

In [ ]:
val result_pr = results.vertices.select("id", "pagerank")
val popular_venues = result_pr.
    join(venues, result_pr("id") === venues("vid"), "inner").
    select("vid", "pagerank", "name")

popular_venues.sort($"pagerank".desc).show(13, false)

As shown above, this algorithm provides a "popularity" factor for each checked-in venue. This feature can be used to further discriminate anomalies based on the rank of the venue; for instance, combining it with the probability of checking in at a specific time of the day.

### Geolocation: density based

We will now cluster events based on the <a href="https://en.wikipedia.org/wiki/DBSCAN" target="_blank">DBSCAN algorithm</a>. DBSCAN is clustering events depending on the density of the events provided. Since the clusters emerge locally by looking for neighboring points, clusters of various shapes can be detected. Points that are isolated and too far from any other point are assigned to a special cluster of outliers. These discerning properties make the DBSCAN algorithm a good candidate for clustering geolocated events.

Let's prepare the data by transforming the events DataFrame, into a PairRDD. In particular, for geolocated data, we choose the key to be the user identifier, and the value to be the aggregated list of all check-ins posted by that given user. The geolocated data is arranged in an n-by-2 matrix, where the first column represents the latitude and the second column represents the longitude. 

In [ ]:
val top_users = events.
    groupBy($"uid").
    count().
    filter($"count" > 1000)
    
top_users.sort(desc("count")).show()

In [ ]:
val top_events = events.
    join(top_users, top_users("uid")  === events("uid"), "inner").
    select("ts", "events.uid", "lat", "lon")

top_events.show(5, false)

#### From long to wide

What we would like to do now, as a preparation for the DBSCAN clustering algorithm is to take all events registered by each user and store them as a single array of tuples. One way to do that is to create a key-value RDD (check <a href="http://spark.apache.org/docs/latest/programming-guide.html#working-with-key-value-pairs" target="_blank">http://spark.apache.org/docs/latest/programming-guide.html#working-with-key-value-pairs</a>), where the key is the user id and the value is an array of co-ordinates tuples. How to go wide? We concatenate vectors using the `reduceByKey` rdd function, in this way we can redistribute the data in a format which works well for our DBSCAN algorithm.


In [ ]:
import breeze.linalg._
import breeze.linalg.DenseMatrix

In [ ]:
val e_df = top_events.
  select("uid","lat","lon").
  rdd.map(row => (row.getLong(0), Array(row.getDouble(1), row.getDouble(2))) ).
  reduceByKey( _ ++ _).
  mapValues(v => new DenseMatrix(v.length/2,2,v, 0, 2, true))


What we have just created is a complex data structure; for debug and visualization, let's create a well formatted printing utility

In [ ]:
def formatUserEvents(x: Tuple2[Long, DenseMatrix[Double]]) : Unit = {
    val arr = x._2
    val n = math.min( 5 , arr.rows) - 1
    val slice = arr(0 to n, ::)
    println(s"uid = ${x._1}")
    println(s"events count = ${arr.rows}")
    println("lat,lon = ")
    println(slice)
    if (arr.rows > 5) println(s"... ${arr.rows- 5} more rows")
    println("-"*30)
}

Now we can use the formatting function, with Spark and Scala foreach statements.  
Below, you will see a formatted output describing the events related to three users:

In [ ]:
e_df.take(3).foreach(e => formatUserEvents(e))

We will now cluster the events for each user according to the DBSCAN algorithm. This algorithm will cluster those users' events in groups. The rest of the code below reduces those groups to bounding boxes. Next, we will use the extracted bounding boxes to score events.

In [ ]:
import breeze.numerics._
import breeze.linalg._

def euclideanDistance (a: DenseVector[Double], b: DenseVector[Double]) = norm(a-b, 2)

// 1deg at 40deg latitude is 111034.61 meters
// set radius at about 200 mt (0.002 * 111034.61)
// which is 0.002 in decimal degrees https://en.wikipedia.org/wiki/Decimal_degrees

val eps = 0.002
val min_points = 3

In [ ]:
import nak.cluster._
import nak.cluster.GDBSCAN._

def dbscan(v : breeze.linalg.DenseMatrix[Double]) = {

  val gdbscan = new GDBSCAN(
    DBSCAN.getNeighbours(eps, distance=euclideanDistance),
    DBSCAN.isCorePoint(min_points)
  )

  // core DBSCAN algorithm
  val clusters = gdbscan cluster v
  
  // reducing the clusters to bounding boxes
  // for simplicity: each user could 
  clusters.map(
    cluster => (
      cluster.id.toInt, 
      cluster.points.size, 
      cluster.points.map(_.value(0)).min,
      cluster.points.map(_.value(1)).min,
      cluster.points.map(_.value(0)).max,
      cluster.points.map(_.value(1)).max
    )
  )
}

Next, is what is probably the most powerful one-liner in this tutorial.   
It will stage the DBSCAN clustering algorithm for all users and their respective events.

In [ ]:
val bboxRdd = e_df.mapValues(dbscan(_)).cache()

Let's convert the RDDs back to a DataFrame. Now we have a table describing clusters. Each row defines a cluster in terms of user id, cluster id, the number of clusters' events, and the bounding box of the cluster. Each user can have multiple clusters, and some users might have no cluster at all.

In [ ]:
val bbox_df = bboxRdd.
  flatMapValues(x => x).
  map(x => (x._1, x._2._1, x._2._2,x._2._3,x._2._4,x._2._5,x._2._6)).
  toDF("uid", "cid", "csize", "lat_min", "lon_min", "lat_max", "lon_max").
  filter($"cid" > 0)

bbox_df.show(10)

#### Scoring events: looking for outliers

We will now score events and look if some of them are located outside the computed clusters' bounding boxes. First, we join the table of events with the table of clusters. Then, let's filter out users which do not have enough points, as those users have no clusters associated with them and there is not sufficient data to determine outliers. In the code above, we need a user to have at least 3 events in a region of 0.1 degrees in order to have a DBSCAN cluster.

In [ ]:
val bbox_events = events.
  join(bbox_df, events("events.uid") === bbox_df("uid"), "full").
  select($"events.ts",$"events.uid",$"lat",$"lon",$"lat_min",$"lon_min",$"lat_max",$"lon_max").
  filter($"lat_min".isNotNull)

bbox_events.show(5,false)

In [ ]:
import java.sql.Timestamp

case class EventBbox(
  ts: Timestamp,
  uid: Long, 
  lat:Double, 
  lon: Double, 
  lat_min:Double, 
  lon_min:Double, 
  lat_max:Double, 
  lon_max:Double)
  
case class EventDetected(
  ts: Timestamp,
  uid: Long, 
  lat: Double, 
  lon: Double, 
  bbox: Boolean
)

In [ ]:
def bbox_check( x:EventBbox): Boolean = {
  x.lon >= x.lon_min &
  x.lon <= x.lon_max &
  x.lat >= x.lat_min &
  x.lat <= x.lat_max   
}

The following code uses the newer Dataset API, which is a DataFrame where rows are handled as typed objects. In particular, we are converting the events row into a `EventDetected` object and then we check if the event is within the boundary of the given cluster. Since each user might have more than one cluster, we check each event against all the user's clusters and we consider it an outlier if none of the checked return a positive outcome. 

In [ ]:
val scored_events = bbox_events.
  as[EventBbox].
  map(x => EventDetected(x.ts, x.uid, x.lat, x.lon, bbox_check(x))).
  groupBy($"ts", $"uid").
  reduce( (x,y) => if (x.bbox) x else y ).
  map(x => x._2).
  cache()

Below is the outlier scoring for `uid=22`. 

In [ ]:
scored_events.
  filter(_.uid==22).
  show(10)

As you can see, three items are found outside those bounding boxes. Although this is not yet a strong indicator for an anomaly per se, it can constitute a very relevant signal if combined with other signals as seen above. Many improvements can be made to the above core idea; for instance, by including relations and interaction between users and more refined analysis of clusters, using convex hulls instead of bounding boxes, and so forth.

I hope you enjoyed this notebook, thanks for keeping up with me. Best wishes for your data science projects!